In [ ]:
! pip install langchain

In [1]:
# Run this first!
from langchain.chat_models import ChatOpenAI
api_key = open("key_openai.txt", "r").read().strip()
llm = ChatOpenAI(model_name='gpt-4', openai_api_key=api_key)
math_problem1 = "Completely simplify and rationalize the denominator: $$\\frac{\\sqrt{160}}{\\sqrt{252}}\\times\\frac{\\sqrt{245}}{\\sqrt{108}}$$"
math_problem2 = "There are 190 people on the beach.  110 are wearing sunglasses, 70 are wearing bathing suits, and 95 are wearing a hat.  Everyone is wearing at least one of these items. 30 are wearing both bathing suits and sunglasses. 25 are wearing both bathing suits and a hat. 40 are wearing both sunglasses and a hat.  How many people are wearing all three items?" # fill the problem here


In [2]:
from langchain.agents import initialize_agent
from langchain.agents import AgentType
from langchain.tools.python.tool import PythonREPLTool

tools = [PythonREPLTool()]
agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)
result = agent.run(math_problem2)



> Entering new AgentExecutor chain...


Python REPL can execute arbitrary code. Use with caution.


We will use principle of inclusion and exclusion which states that for any three sets A, B and C, the number of elements in their union can be found using the formula |A ∪ B ∪ C| = |A| + |B| + |C| - |A ∩ B| - |A ∩ C| - |B ∩ C| + |A ∩ B ∩ C|. 

Given are:
|A| = number of people wearing sunglasses = 110
|B| = number of people wearing bathing suits = 70
|C| = number of people wearing a hat = 95

|A ∪ B ∪ C| = total people on the beach = 190
|A ∩ B| = people wearing both bathing suits and sunglasses = 30
|A ∩ C| = people wearing both sunglasses and a hat = 40
|B ∩ C| = people wearing both bathing suits and a hat = 25

We need to find |A ∩ B ∩ C| = people wearing all three items.

We can rearrange the formula to solve for |A ∩ B ∩ C|:

|A ∩ B ∩ C| = |A| + |B| + |C| - |A ∩ B| - |A ∩ C| - |B ∩ C| + |A ∪ B ∪ C|
Action: Python_REPL
Action Input: A = 110
B = 70
C = 95
A_union_B_union_C = 190
A_intersect_B = 30
A_intersect_C = 40
B_intersect_C = 25

A_intersect_B_intersect_C = A + B + C - A_inter

OutputParserException: Could not parse LLM output: `The calculation is still incorrect. The result cannot be a negative number. I realize now that I've been subtracting the total number of people on the beach from the calculation, which is incorrect. The total number of people on the beach is the sum of all of the individual and overlapping categories, so it does not need to be subtracted. The correct formula should be:

|A ∩ B ∩ C| = |A| + |B| + |C| - |A ∩ B| - |A ∩ C| - |B ∩ C| + |A ∪ B ∪ C|

I need to correct the equation and calculate it again.`

NameError: name 'result' is not defined

## Two problems are tested

Problem 1: Completely simplify and rationalize the denominator: $$\frac{\sqrt{160}}{\sqrt{252}}\times\frac{\sqrt{245}}{\sqrt{108}}$$  
Answer: $$\boxed{\frac{5\sqrt{42}}{27}}$$


Problem 2: There are 190 people on the beach.  110 are wearing sunglasses, 70 are wearing bathing suits, and 95 are wearing a hat.  Everyone is wearing at least one of these items. 30 are wearing both bathing suits and sunglasses. 25 are wearing both bathing suits and a hat. 40 are wearing both sunglasses and a hat.  How many people are wearing all three items?    
Answer: $$\boxed{10}$$

## [Symbolic Math Chain](https://python.langchain.com/docs/modules/chains/additional/llm_symbolic_math)

Tested on 2 problems, one is correct and the other is wrong.

In [10]:
# symbolic math
from langchain.chains.llm_symbolic_math.base import LLMSymbolicMathChain
llm_symbolic_math = LLMSymbolicMathChain.from_llm(llm)

llm_symbolic_math.run(math_problem1)

'Answer: 5*sqrt(42)/27'

In [11]:
llm_symbolic_math.run(math_problem2)

'Answer: Eq(190, x + 180)'

**Another run of math_problem2:**   
ValueError: Error from parse_expr with transformed code: "Symbol ('Eq1' )=Eq (Symbol ('x' )+Symbol ('y' )+Symbol ('z' ),Integer (110 ))# sunglassesEq2 = Eq(x + w + z, 70)   # bathing suitsEq3 = Eq(w + y + z, 95)   # hatEq4 = Eq(x + z, 30)       # bathing suits and sunglassesEq5 = Eq(w + z, 25)       # bathing suits and hatEq6 = Eq(y + z, 40)       # sunglasses and hatsolve((Eq1, Eq2, Eq3, Eq4, Eq5, Eq6), (x, y, z, w))"

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/home/ykw5399/.conda/envs/camel/lib/python3.10/site-packages/sympy/core/sympify.py", line 495, in sympify
    expr = parse_expr(a, local_dict=locals, transformations=transformations, evaluate=evaluate)
  File "/home/ykw5399/.conda/envs/camel/lib/python3.10/site-packages/sympy/parsing/sympy_parser.py", line 1087, in parse_expr
    raise e from ValueError(f"Error from parse_expr with transformed code: {code!r}")
  File "/home/ykw5399/.conda/envs/camel/lib/python3.10/site-packages/sympy/parsing/sympy_parser.py", line 1078, in parse_expr
    rv = eval_expr(code, local_dict, global_dict)
  File "/home/ykw5399/.conda/envs/camel/lib/python3.10/site-packages/sympy/parsing/sympy_parser.py", line 906, in eval_expr
    expr = eval(
  File "<string>", line 1
    Symbol ('Eq1' )=Eq (Symbol ('x' )+Symbol ('y' )+Symbol ('z' ),Integer (110 ))# sunglassesEq2 = Eq(x + w + z, 70)   # bathing suitsEq3 = Eq(w + y + z, 95)   # hatEq4 = Eq(x + z, 30)       # bathing suits and sunglassesEq5 = Eq(w + z, 25)       # bathing suits and hatEq6 = Eq(y + z, 40)       # sunglasses and hatsolve((Eq1, Eq2, Eq3, Eq4, Eq5, Eq6), (x, y, z, w))
                   ^
SyntaxError: invalid syntax

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/ykw5399/.conda/envs/camel/lib/python3.10/site-packages/langchain/chains/llm_symbolic_math/base.py", line 64, in _evaluate_expression
    output = str(sympy.sympify(expression, evaluate=True))
  File "/home/ykw5399/.conda/envs/camel/lib/python3.10/site-packages/sympy/core/sympify.py", line 497, in sympify
    raise SympifyError('could not parse %r' % a, exc)
sympy.core.sympify.SympifyError: Sympify of expression 'could not parse 'Eq1 = Eq(x + y + z, 110)  # sunglassesEq2 = Eq(x + w + z, 70)   # bathing suitsEq3 = Eq(w + y + z, 95)   # hatEq4 = Eq(x + z, 30)       # bathing suits and sunglassesEq5 = Eq(w + z, 25)       # bathing suits and hatEq6 = Eq(y + z, 40)       # sunglasses and hatsolve((Eq1, Eq2, Eq3, Eq4, Eq5, Eq6), (x, y, z, w))'' failed, because of exception being raised:
SyntaxError: invalid syntax (<string>, line 1)

During handling of the above exception, another exception occurred:

... More errors not shown

## [PAL Chain](https://python.langchain.com/docs/modules/chains/additional/pal)

Tested on 2 problems, got one error and one wrong answer.

In [12]:
from langchain.chains import PALChain
pal_chain = PALChain.from_math_prompt(llm, verbose=True)
pal_chain.run(math_problem1)



> Entering new PALChain chain...
import math


ValueError: Generated code is missing the solution expression: solution of type: <class 'ast.FunctionDef'>

**Previous Result:**   
 Entering new PALChain chain...
def solution():
    """Find all $x$ that satisfy the inequality $(2x+10)(x+3)<(3x+9)(x+8)$. Express your answer in interval notation."""
    from sympy import *
    from sympy.abc import x
Traceback (most recent call last):
  File "/home/ykw5399/msft/FLAML/evaluation/math/test_math_chains.py", line 17, in <module>
    print(pal_chain.run(math_problem))
  File "/home/ykw5399/.conda/envs/camel/lib/python3.10/site-packages/langchain/chains/base.py", line 440, in run
    return self(args[0], callbacks=callbacks, tags=tags, metadata=metadata)[
  File "/home/ykw5399/.conda/envs/camel/lib/python3.10/site-packages/langchain/chains/base.py", line 243, in __call__
    raise e
  File "/home/ykw5399/.conda/envs/camel/lib/python3.10/site-packages/langchain/chains/base.py", line 237, in __call__
    self._call(inputs, run_manager=run_manager)
  File "/home/ykw5399/.conda/envs/camel/lib/python3.10/site-packages/langchain/chains/pal/base.py", line 166, in _call
    PALChain.validate_code(code, self.code_validations)
  File "/home/ykw5399/.conda/envs/camel/lib/python3.10/site-packages/langchain/chains/pal/base.py", line 261, in validate_code
    raise ValueError(f"Generated code has disallowed imports: {code}")
ValueError: Generated code has disallowed imports: def solution():
    """Find all $x$ that satisfy the inequality $(2x+10)(x+3)<(3x+9)(x+8)$. Express your answer in interval notation."""
    from sympy import *
    from sympy.abc import x

In [13]:
pal_chain.run(math_problem2)



> Entering new PALChain chain...
def solution():
    """There are 190 people on the beach.  110 are wearing sunglasses, 70 are wearing bathing suits, and 95 are wearing a hat.  Everyone is wearing at least one of these items. 30 are wearing both bathing suits and sunglasses. 25 are wearing both bathing suits and a hat. 40 are wearing both sunglasses and a hat.  How many people are wearing all three items?"""
    total_people = 190
    sunglasses = 110
    bathing_suits = 70
    hats = 95
    both_sunglasses_bathing_suits = 30
    both_bathing_suits_hats = 25
    both_sunglasses_hats = 40

> Finished chain.


'None'

**Another run of math_problem2:** 
Entering new PALChain chain...
def solution():
    """There are 190 people on the beach.  110 are wearing sunglasses, 70 are wearing bathing suits, and 95 are wearing a hat.  Everyone is wearing at least one of these items. 30 are wearing both bathing suits and sunglasses. 25 are wearing both bathing suits and a hat. 40 are wearing both sunglasses and a hat.  How many people are wearing all three items?"""
    total_people = 190
    sunglasses = 110
    bathing_suits = 70
    hats = 95
    both_bathing_suits_sunglasses = 30
    both_bathing_suits_hats = 25
    both_sunglasses_hats = 40
    all_three = (sunglasses + bathing_suits + hats) - (both_bathing_suits_sunglasses + both_bathing_suits_hats + both_sunglasses_hats)
    result = all_three
    return result
Python REPL can execute arbitrary code. Use with caution.

Finished chain.
180

The correct answer is 10.

##  [math chain](https://python.langchain.com/docs/modules/chains/additional/llm_math)

Mathchain got the first problem correct and the second problem wrong.

In [14]:
from langchain import LLMMathChain

llm_math = LLMMathChain.from_llm(llm, verbose=True)
llm_math.run(math_problem1)



> Entering new LLMMathChain chain...
Completely simplify and rationalize the denominator: $$rac{\sqrt{160}}{\sqrt{252}}	imesrac{\sqrt{245}}{\sqrt{108}}$$```text
(sqrt(160)/sqrt(252)) * (sqrt(245)/sqrt(108))
```
...numexpr.evaluate("(sqrt(160)/sqrt(252)) * (sqrt(245)/sqrt(108))")...

Answer: 1.2001371663718259
> Finished chain.


'Answer: 1.2001371663718259'

In [16]:
llm_math.run(math_problem2)



> Entering new LLMMathChain chain...
There are 190 people on the beach.  110 are wearing sunglasses, 70 are wearing bathing suits, and 95 are wearing a hat.  Everyone is wearing at least one of these items. 30 are wearing both bathing suits and sunglasses. 25 are wearing both bathing suits and a hat. 40 are wearing both sunglasses and a hat.  How many people are wearing all three items?```text
190 - 110 - 70 - 95 + 30 + 25 + 40
```
...numexpr.evaluate("190 - 110 - 70 - 95 + 30 + 25 + 40")...

Answer: 10
> Finished chain.


'Answer: 10'

## Test ReAct + PythonREPL
Langchain agent for ReAct example: https://python.langchain.com/docs/modules/agents/agent_types/react    
ReAct: https://react-lm.github.io/

ReAct got the first problem correct and the second problem wrong.

In [7]:
from langchain.agents import initialize_agent
from langchain.agents import AgentType
from langchain.tools.python.tool import PythonREPLTool

tools = [PythonREPLTool()]
agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)
agent.run(math_problem2)



> Entering new AgentExecutor chain...


First, I will simplify each square root separately. Then I will multiply the numerators together and the denominators together. If the result is not a rational number, I will rationalize the denominator. 
Action: Python_REPL
Action Input: 
```python
import math

# simplifying square roots
sqrt_160 = math.sqrt(160)
sqrt_252 = math.sqrt(252)
sqrt_245 = math.sqrt(245)
sqrt_108 = math.sqrt(108)

# multiplying numerators and denominators
numerator = sqrt_160 * sqrt_245
denominator = sqrt_252 * sqrt_108

print(numerator, denominator)
```
Observation: 197.98989873223334 164.97272501841024

Thought:I have simplified the square roots and multiplied the numerators and denominators together. Now, I will divide the numerator by the denominator to simplify the expression further.
Action: Python_REPL
Action Input: 
```python
# divide numerator by denominator
result = numerator / denominator

print(result)
```
Observation: 1.200137166371826

Thought:The result is a rational number, so there is no nee

'1.200137166371826'

In [15]:
agent.run(math_problem2)



> Entering new AgentExecutor chain...
This is a question about sets and their intersections. The total number of people on the beach is the union of the three sets: people wearing sunglasses, people wearing bathing suits and people wearing a hat. The intersections of these sets are the people wearing two of these items. The intersection of all three sets is the people wearing all three items. We need to find this last number.
To solve this, we can use the principle of inclusion and exclusion, which states that for three sets A, B and C, the number of elements in the union of the sets is equal to the sum of the sizes of the sets minus the sum of the sizes of their pairwise intersections, plus the size of their triple intersection.
Let's denote the number of people wearing sunglasses as S, the number of people wearing bathing suits as B, the number of people wearing a hat as H, the number of people wearing both bathing suits and sunglasses as BS, the number of people wearing both bathi

'10'